Generate Fake Data and Insert rows into OLTP function

In [1]:
#def generate_and_insert_data(num_rows)
import sqlite3
from faker import Faker
import random
from datetime import timedelta

def generate_and_insert_data(num_rows):
    conn = sqlite3.connect('newhotel.db')
    c = conn.cursor()

    # Get the maximum IDs from the tables
    max_guest_id = c.execute('SELECT MAX(GuestID) FROM guests').fetchone()[0] or 0
    max_room_id = c.execute('SELECT MAX(RoomID) FROM rooms').fetchone()[0] or 0
    max_booking_id = c.execute('SELECT MAX(BookingID) FROM bookings').fetchone()[0] or 0
    max_employee_id = c.execute('SELECT MAX(EmployeeID) FROM employees').fetchone()[0] or 0
    max_service_id = c.execute('SELECT MAX(ServiceID) FROM services').fetchone()[0] or 0
    max_service_booking_id = c.execute('SELECT MAX(ServiceBookingID) FROM servicebookings').fetchone()[0] or 0
    max_payment_id = c.execute('SELECT MAX(PaymentID) FROM payments').fetchone()[0] or 0
    max_review_id = c.execute('SELECT MAX(ReviewID) FROM reviews').fetchone()[0] or 0
    max_maintenance_id = c.execute('SELECT MAX(MaintenanceID) FROM roommaintenance').fetchone()[0] or 0

    fake = Faker()

    # Data Lists
    guests = []
    rooms = []
    bookings = []
    employees = []
    services = []
    service_bookings = []
    payments = []
    reviews = []
    room_maintenance = []

        # Fill data lists
    for i in range(num_rows):
        guests.append((max_guest_id + i + 1, fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(),
                    str(fake.date_of_birth(minimum_age=18, maximum_age=85)), fake.country(), random.choice([True, False])))
        rooms.append((max_room_id + i + 1, random.choice(['Single', 'Double', 'Suite']), round(random.uniform(100, 400), 2),
                    random.choice(['Available', 'Occupied', 'Maintenance']), random.randint(1, 10)))

        check_in = fake.date_this_year()
        check_out = check_in + timedelta(days=random.randint(1, 5))
        bookings.append((max_booking_id + i + 1, random.randint(1, num_rows), random.randint(1, num_rows), str(check_in), str(check_out),
                        random.randint(1, 4), round(random.uniform(100.0, 1000.0), 2), round(random.uniform(5.0, 100.0), 2)))

        employees.append((max_employee_id + i + 1, fake.first_name(), fake.last_name(), random.choice(['Manager', 'Receptionist', 'Housekeeper', 'Chef']),
                        str(fake.date_this_decade()), round(random.uniform(30000, 60000), 2), random.choice(['Administration', 'Housekeeping', 'Kitchen', 'Front Desk']),
                        random.randint(1, 10)))

        services.append((max_service_id + i + 1, fake.word(), fake.word(), round(random.uniform(20, 100), 2), round(random.uniform(10, 50), 2), fake.time()))

        service_bookings.append((max_service_booking_id + i + 1, random.randint(1, num_rows), random.randint(1, num_rows), str(fake.date_this_year()), fake.time(), random.randint(1, 5),
                                round(random.uniform(20.0, 200.0), 2)))

        payments.append((max_payment_id + i + 1, random.randint(1, num_rows), str(fake.date_this_year()), random.choice(['Cash', 'Credit', 'Debit']),
                    round(random.uniform(100.0, 1000.0), 2), round(random.uniform(5.0, 100.0), 2), round(random.uniform(100.0, 150.0), 2), round(random.uniform(0.0, 50.0), 2)))

        reviews.append((max_review_id + i + 1, random.randint(1, num_rows), random.randint(1, 5), fake.text(), fake.text(), random.randint(1, 5)))

        room_maintenance.append((max_maintenance_id + i + 1, random.randint(1, num_rows), random.randint(1, num_rows), str(fake.date_this_year()), str(fake.date_this_year()),
                                fake.word(), random.choice(['Pending', 'In Progress', 'Completed']), round(random.uniform(50, 500), 2), random.randint(1, 24)))

        
        # Connect to the SQLite database
    with sqlite3.connect('newhotel.db') as conn:
        c = conn.cursor()

        # Insert data into the database
        for guest in guests:
            c.execute("INSERT INTO guests VALUES (?, ?, ?, ?, ?, ?, ?, ?)", guest)
        for room in rooms:
            c.execute("INSERT INTO rooms VALUES (?, ?, ?, ?, ?)", room)
        for booking in bookings:
            c.execute("INSERT INTO bookings VALUES (?, ?, ?, ?, ?, ?, ?, ?)", booking)
        for employee in employees:
            c.execute("INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?, ?, ?)", employee)
        for service in services:
            c.execute("INSERT INTO services VALUES (?, ?, ?, ?, ?, ?)", service)
        for service_booking in service_bookings:
            c.execute("INSERT INTO servicebookings VALUES (?, ?, ?, ?, ?, ?, ?)", service_booking)
        for payment in payments:
            c.execute("INSERT INTO payments VALUES (?, ?, ?, ?, ?, ?, ?, ?)", payment)
        for review in reviews:
            c.execute("INSERT INTO reviews VALUES (?, ?, ?, ?, ?, ?)", review)
        for maintenance in room_maintenance:
            c.execute("INSERT INTO roommaintenance VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", maintenance)

        # Commit changes
        conn.commit()

   

In [42]:
#check if rows got inserted 
def count_rows():
    conn = sqlite3.connect('newhotel.db')
    c = conn.cursor()

    tables = ['guests', 'rooms', 'bookings', 'employees', 'services', 'servicebookings', 'payments', 'reviews', 'roommaintenance']

    for table in tables:
        count = c.execute(f'SELECT COUNT(*) FROM {table}').fetchone()[0]
        print(f'The table {table} has {count} rows.')

    conn.close()

# Call the function
count_rows()

The table guests has 4650 rows.
The table rooms has 4650 rows.
The table bookings has 4650 rows.
The table employees has 4650 rows.
The table services has 4650 rows.
The table servicebookings has 4650 rows.
The table payments has 4650 rows.
The table reviews has 4650 rows.
The table roommaintenance has 4649 rows.


Staging Area Ingestion function (data frames)

In [3]:
#def create_staging_area():

import pandas as pd
import sqlite3

def create_staging_area():
    conn = sqlite3.connect('newhotel.db')

    # Read data from guests table into a DataFrame
    df_guests = pd.read_sql_query("SELECT guestid, firstname, lastname, country, vipstatus, DateOfBirth FROM guests", conn)

   # Read data from bookings and rooms tables into a DataFrame
    df_dimbooking = pd.read_sql_query("""
        SELECT bookings.bookingid, bookings.guestid, bookings.roomid, rooms.roomtype, rooms.pricepernight, bookings.numberofguests, bookings.discounts, bookings.totalbill, bookings.checkindate, bookings.checkoutdate
        FROM bookings 
        JOIN rooms ON bookings.roomid = rooms.roomid
    """, conn)

    # Read data from services and service bookings tables into a DataFrame
    df_servicesbooking = pd.read_sql_query("""
        SELECT ServiceBookings.ServiceBookingID, ServiceBookings.BookingID, ServiceBookings.ServiceID, ServiceBookings.Date, Services.ServiceType, Services.ServiceName, Services.Price, Services.Cost, ServiceBookings.TotalServiceCost
        FROM ServiceBookings
        JOIN Services ON ServiceBookings.ServiceID = Services.ServiceID
    """, conn)

    # Read data from payments table into a DataFrame
    df_payments = pd.read_sql_query("""
    SELECT PaymentID, BookingID, PaymentMethod, PaymentDate, AmountPaid, AmountDue, Revenue
    FROM Payments
    """, conn)

    # Create a new DataFrame for the fact table
    df_fact = pd.DataFrame()

    # Copy guestid from df_guests into df_fact
    df_fact['GuestID'] = df_guests['GuestID'].copy()

    # Copy required columns from df_dimbooking into df_fact
    df_fact['BookingID'] = df_dimbooking['BookingID'].copy()
    df_fact['PricePerNight'] = df_dimbooking['PricePerNight'].copy()
    df_fact['NumberOfGuests'] = df_dimbooking['NumberOfGuests'].copy()
    df_fact['Discounts'] = df_dimbooking['Discounts'].copy()
    df_fact['TotalBill'] = df_dimbooking['TotalBill'].copy()

    # Drop specified columns from df_dimbooking
    df_dimbooking = df_dimbooking.drop(['PricePerNight', 'NumberOfGuests', 'Discounts', 'TotalBill'], axis=1)

    # Copy required columns from df_servicesbooking into df_fact
    df_fact['ServiceBookingID'] = df_servicesbooking['ServiceBookingID'].copy()
    df_fact['Price'] = df_servicesbooking['Price'].copy()
    df_fact['Cost'] = df_servicesbooking['Cost'].copy()
    df_fact['TotalServiceCost'] = df_servicesbooking['TotalServiceCost'].copy()

    # Drop specified columns from df_servicesbooking
    df_servicesbooking = df_servicesbooking.drop(['Price', 'Cost', 'TotalServiceCost'], axis=1)

    # Copy required columns from df_payments into df_fact
    df_fact['PaymentID'] = df_payments['PaymentID'].copy()
    df_fact['AmountDue'] = df_payments['AmountDue'].copy()
    df_fact['AmountPaid'] = df_payments['AmountPaid'].copy()
    df_fact['Revenue'] = df_payments['Revenue'].copy()

    # Drop specified columns from df_payments
    df_payments = df_payments.drop(['AmountDue', 'AmountPaid', 'Revenue'], axis=1)

    # Close connection
    conn.close()

    return df_guests, df_dimbooking, df_servicesbooking, df_payments, df_fact

# Call the function and print the DataFrame
df_guests, df_dimbooking, df_servicesbooking, df_payments, df_fact = create_staging_area()


Cleaning Function (Imputes missing values using mean and mode)

In [5]:
def clean_df(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].mean())
        elif pd.api.types.is_string_dtype(df[col]):
            df[col] = df[col].fillna(df[col].mode()[0])
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = pd.to_datetime(df[col], errors='coerce')
            if df[col].isnull().any():
                print(f"Column {col} could not be converted to datetime.")
    return df

Creation of New Attribute

In [6]:
def add_stay_duration(df):
    # Convert CheckInDate and CheckOutDate to datetime
    df['CheckInDate'] = pd.to_datetime(df['CheckInDate'], format='%Y-%m-%d', errors='coerce')
    df['CheckOutDate'] = pd.to_datetime(df['CheckOutDate'], format='%Y-%m-%d', errors='coerce')

    # Check if conversion was successful
    if df['CheckInDate'].isnull().any() or df['CheckOutDate'].isnull().any():
        print("CheckInDate or CheckOutDate could not be converted to datetime.")
        return df

    # Create StayDuration attribute
    df['StayDuration'] = (df['CheckOutDate'] - df['CheckInDate']).dt.days

    return df

Mapping Data From Staging Area to Data Mart

In [7]:
#Mapping function for each dimension
import sqlite3

def map_to_dimGuests(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute("SELECT * FROM dimGuests WHERE GuestID = ?", (row['GuestID'],))
        data = cur.fetchone()
        if data is None:
            cur.execute("INSERT INTO dimGuests VALUES (?, ?, ?, ?, ?, ?)", 
                        (row['GuestID'], row['FirstName'], row['LastName'], row['Country'], row['VIPStatus'], row['DateOfBirth']))
    conn.commit()

def map_to_dimBooking(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute("SELECT * FROM dimBooking WHERE BookingID = ?", (row['BookingID'],))
        data = cur.fetchone()
        if data is None:
            # Convert 'CheckInDate' and 'CheckOutDate' to string format
            check_in_date = row['CheckInDate'].strftime('%Y-%m-%d %H:%M:%S')
            check_out_date = row['CheckOutDate'].strftime('%Y-%m-%d %H:%M:%S')
            cur.execute("INSERT INTO dimBooking VALUES (?, ?, ?, ?, ?, ?, ?)", 
                        (row['BookingID'], row['GuestID'], row['RoomID'], row['RoomType'], check_in_date, check_out_date, row['StayDuration']))
    conn.commit()

def map_to_dimServicesBooking(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute("SELECT * FROM dimServicesBooking WHERE ServiceBookingID = ?", (row['ServiceBookingID'],))
        data = cur.fetchone()
        if data is None:
            cur.execute("INSERT INTO dimServicesBooking VALUES (?, ?, ?, ?, ?, ?)", 
                        (row['ServiceBookingID'], row['BookingID'], row['ServiceID'], row['Date'], row['ServiceType'], row['ServiceName']))
    conn.commit()

def map_to_dimPayments(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute("SELECT * FROM dimPayments WHERE PaymentID = ?", (row['PaymentID'],))
        data = cur.fetchone()
        if data is None:
            cur.execute("INSERT INTO dimPayments VALUES (?, ?, ?, ?)", 
                        (row['PaymentID'], row['BookingID'], row['PaymentMethod'], row['PaymentDate']))
    conn.commit()

def map_to_factTable(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute("SELECT * FROM factTable WHERE BookingID = ? AND PaymentID = ?", (row['BookingID'], row['PaymentID']))
        data = cur.fetchone()
        if data is None:
            cur.execute("INSERT INTO factTable VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                        (row['BookingID'], row['PaymentID'], row['GuestID'], row['ServiceBookingID'], row['PricePerNight'], row['NumberOfGuests'], row['TotalBill'], row['Discounts'], row['Price'], row['Cost'], row['TotalServiceCost'], row['AmountPaid'], row['AmountDue'], row['Revenue']))
    conn.commit()

Generate Snapshot Function

In [9]:
def generate_fact_table_snapshot(conn):
    # Query to join the fact table with the four dimension tables
    query = """
    SELECT *
    FROM factTable
    LEFT JOIN dimGuests ON factTable.GuestID = dimGuests.GuestID
    LEFT JOIN dimBooking ON factTable.BookingID = dimBooking.BookingID
    LEFT JOIN dimServicesBooking ON factTable.ServiceBookingID = dimServicesBooking.ServiceBookingID
    LEFT JOIN dimPayments ON factTable.PaymentID = dimPayments.PaymentID
    """
    
    # Execute the query and store the result in a DataFrame
    df_fact_table_snapshot = pd.read_sql_query(query, conn)
    
    return df_fact_table_snapshot

Create Dashboard Function and % Change Dashboard Function

In [27]:
import pandas as pd
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

def create_dashboard(df_snapshot, old_snapshot_filename='old_snapshot.csv'):
    # Load the old snapshot
    try:
        df_old_snapshot = pd.read_csv(old_snapshot_filename)
    except FileNotFoundError:
        df_old_snapshot = pd.DataFrame()

    # Create a Dash app
    app = dash.Dash(__name__)

    # Calculate total revenue for each country and get the top 10
    top_countries = df_snapshot.groupby('Country')['Revenue'].sum().nlargest(10).reset_index()

    # If old snapshot exists, calculate the percentage change for each country
    if not df_old_snapshot.empty:
        old_top_countries = df_old_snapshot.groupby('Country')['Revenue'].sum().nlargest(10).reset_index()
        top_countries = top_countries.merge(old_top_countries, on='Country', how='left', suffixes=('_new', '_old'))
        top_countries['Revenue_change'] = ((top_countries['Revenue_new'] - top_countries['Revenue_old']) / top_countries['Revenue_old']) * 100

    # Calculate total revenue
    total_revenue = df_snapshot['Revenue'].sum()

    # Calculate the change in total revenue
    if not df_old_snapshot.empty:
        old_total_revenue = df_old_snapshot['Revenue'].sum()
        revenue_change = ((total_revenue - old_total_revenue) / old_total_revenue) * 100
    else:
        revenue_change = 0

    # Create a bar chart for the total revenue for the top 10 countries
    fig1 = px.bar(top_countries, x='Country', y='Revenue_new', title='Total Revenue for Top 10 Countries', 
                  labels={'Country': 'Country', 'Revenue_new': 'Total Revenue'}, 
                  color='Country', barmode='group')

    # Create a bar chart for the revenue change for the top 10 countries
    fig2 = px.bar(top_countries, x='Country', y='Revenue_change', title='Revenue Change for Top 10 Countries', 
                  labels={'Country': 'Country', 'Revenue_change': 'Revenue Change (%)'}, 
                  color='Country', barmode='group')

    # Convert 'PaymentDate' to datetime
    df_snapshot['PaymentDate'] = pd.to_datetime(df_snapshot['PaymentDate'])

    # Group by month and sum the revenue
    df_snapshot['Month'] = df_snapshot['PaymentDate'].dt.to_period('M')
    monthly_revenue = df_snapshot.groupby('Month')['Revenue'].sum().reset_index()
    monthly_revenue['Month'] = monthly_revenue['Month'].astype(str)

    # Create a line chart for the monthly revenue
    fig3 = px.line(monthly_revenue, x='Month', y='Revenue', title='Monthly Revenue',
                   labels={'Month': 'Month', 'Revenue': 'Revenue'})

    # Define the layout of the app
    app.layout = html.Div(children=[
        html.H1(children='Hotel Management Dashboard'),

        html.Div(children='''Dash: A web application framework for Python.'''),

        html.H2(children=f'Total Revenue: ${total_revenue:,.2f}'),
        html.H2(children=f'Revenue Change: {revenue_change:.2f}%'),

        dcc.Graph(id='example-graph-1', figure=fig1),

        dcc.Graph(id='example-graph-2', figure=fig2),

        dcc.Graph(id='example-graph-3', figure=fig3)
    ])

    # Run the app
    app.run_server(port=8050)

    # Save the current snapshot for the next execution
    df_snapshot.to_csv(old_snapshot_filename, index=False)

# Example usage
# df_snapshot = pd.read_csv('your_data.csv')
# create_dashboard(df_snapshot)


In [35]:
def master_etl_function(): 
    # Generate and insert data 
    generate_and_insert_data(50) 

    # Staging area
    df_guests, df_dimbooking, df_servicesbooking, df_payments, df_fact = create_staging_area()

    # Call clean_data() 
    df_guests = clean_df(df_guests)
    df_dimbooking = clean_df(df_dimbooking)
    df_servicesbooking = clean_df(df_servicesbooking)
    df_payments = clean_df(df_payments)
    df_fact = clean_df(df_fact)

    # Create new attributes in Booking dimension
    df_dimbooking = add_stay_duration(df_dimbooking)

    import sqlite3

    try:
        # Establish a connection to the SQLite database
        with sqlite3.connect('hmart.db') as conn:
            # Assuming your cleaned DataFrames are named 'df_guests', 'df_booking', 'df_services_booking', 'df_payments', and 'df_fact_table'
            map_to_dimGuests(df_guests, conn)
            map_to_dimBooking(df_dimbooking, conn)
            map_to_dimServicesBooking(df_servicesbooking, conn)
            map_to_dimPayments(df_payments, conn)
            map_to_factTable(df_fact, conn)

        # Create a snapshot of the fact table
        with sqlite3.connect('hmart.db') as conn:
            # Generate the fact table snapshot
            df_fact_table_snapshot = generate_fact_table_snapshot(conn)

        # Call the dashboard creation functions
        create_dashboard(df_fact_table_snapshot)

    except Exception as e:
        print(f"An error occurred: {e}")

In [41]:
master_etl_function()

In [13]:
import sqlite3

def unlock_database(db_path):
    try:
        conn = sqlite3.connect(db_path)
        conn.execute("BEGIN EXCLUSIVE")
        conn.commit()
        print("Database unlocked successfully.")
    except sqlite3.OperationalError:
        print("Database is locked.")
    finally:
        if conn:
            conn.close()

# Use the function
unlock_database('hmart.db')

Database unlocked successfully.


In [34]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('hmart.db')

# Create a cursor object
c = conn.cursor()

# Query the sqlite_master table to get the DDL scripts
c.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = c.fetchall()

# Print the DDL scripts
for table in tables:
    print(f"Table: {table[0]}\nDDL:\n{table[1]}\n")

# Close the connection
conn.close()

Table: dimGuests
DDL:
CREATE TABLE dimGuests (
            GuestID INT PRIMARY KEY,
            FirstName TEXT,
            LastName TEXT,
            Country TEXT,
            VIPStatus TEXT,
            DateOfBirth TEXT
        )

Table: dimBooking
DDL:
CREATE TABLE dimBooking (
            BookingID INT PRIMARY KEY,
            GuestID INT,
            RoomID INT,
            RoomType TEXT,
            CheckInDate TEXT,
            CheckOutDate TEXT,
            StayDuration INT
        )

Table: dimServicesBooking
DDL:
CREATE TABLE dimServicesBooking (
            ServiceBookingID INT PRIMARY KEY,
            BookingID INT,
            ServiceID INT,
            Date TEXT,
            ServiceType TEXT,
            ServiceName TEXT
        )

Table: dimPayments
DDL:
CREATE TABLE dimPayments (
            PaymentID INT PRIMARY KEY,
            BookingID INT,
            PaymentMethod TEXT,
            PaymentDate TEXT
        )

Table: factTable
DDL:
CREATE TABLE factTable (
        